In [2]:
!pip install geopy

In [3]:
import numpy as np
import pandas as pd
import math as m
import matplotlib.pyplot as plt
import seaborn as sns
import json
from geopy.distance import geodesic
import folium

# Indicateur : Nombre d'accidents à vélo

## Base de données accidents

In [4]:
#Nom dossier : 'BaseDeDonnées/Accidents_france/caract-2023.csv'
# Nom dossier : 'BaseDeDonnées/Accidents_france/caract-2022.csv' à rajouter suivant le notebook utilisé google colab ou jupyter
df_accidents_2023 = pd.read_csv("caract-2023.csv",sep=';') #2023
df_accidents_2022 = pd.read_csv("carcteristiques-2022.csv",sep=';')#2022

In [5]:

df_accidents_2022 = df_accidents_2022.rename(columns={'Accident_Id':'Num_Acc'})

In [6]:
#df_vehicules = pd.read_csv('BaseDeDonnées/Accidents_france/vehicles.csv')
# Nom dossier : 'BaseDeDonnées/Accidents_france/vehicules-2023.csv'
#Nom dossier : 'BaseDeDonnées/Accidents_france/vehicules-2022.csv'
df_vehicules_2023 = pd.read_csv('vehicules-2023.csv',sep=';')
df_vehicules_2022 = pd.read_csv('vehicules-2022.csv',sep=';')

## Filtre par département et zone urbaine

In [7]:
df_accidents_grenoble_2023 = df_accidents_2023[df_accidents_2023['dep']=='38'] # 2023
df_accidents_grenoble_2022 = df_accidents_2022[(df_accidents_2022['dep']== '38')] #2022

## Filtre sur les véhicules qui sont des vélos

In [8]:
df_vehicules_2023 = df_vehicules_2023[df_vehicules_2023['catv']==1]
df_vehicules_2022 = df_vehicules_2022[df_vehicules_2022['catv']==1]

## Merge avec Vehicules

In [9]:
df_accidents_grenoble_2023 = pd.merge(df_accidents_grenoble_2023,df_vehicules_2023, how='inner',on='Num_Acc')
df_accidents_grenoble_2022 = pd.merge(df_accidents_grenoble_2022,df_vehicules_2022, how='inner',on='Num_Acc')


In [10]:
df_accidents_grenoble_2023 = df_accidents_grenoble_2023[df_accidents_grenoble_2023['catv']==1]
df_accidents_grenoble_2022 = df_accidents_grenoble_2022[df_accidents_grenoble_2022['catv']==1]


In [11]:
print('Le nombre d\'accidents en 2023 était de : {} et de {} en 2022'.format(len(df_accidents_grenoble_2023),len(df_accidents_grenoble_2022)))

Le nombre d'accidents en 2023 était de : 68 et de 68 en 2022


L'étude de ces fichiers n'est pas terminée mais déjà on peut avoir une idée du nombre d'accidents impliquant des vélos et voir si l'on peut en tirer quelque chose.
Entre 2005 - 2016  , il y a eu 448 accidents de vélo recensés par la police à Grenoble.

## Map des accidents qu'il y eu impliquant un cycliste dans l'agglomération Grenobloise.

In [12]:
df_accidents_grenoble_2023['lat']= df_accidents_grenoble_2023['lat'].str.replace(',','.')
df_accidents_grenoble_2023['long']= df_accidents_grenoble_2023['long'].str.replace(',','.')

In [13]:
map_accidents_carrefour_Grenoble = folium.Map(location=[45.188529, 5.724524],
                                              zoom_start=14,
                                              titles = 'Map Accidents Carrefour Grenoble')


In [14]:
df_accidents_grenoble_2023['lat']=df_accidents_grenoble_2023['lat'].astype(object)
df_accidents_grenoble_2023['long']=df_accidents_grenoble_2023['long'].astype(object)

for index, row in df_accidents_grenoble_2023.iterrows():
    folium.Marker([row['lat'], row['long']],icon=folium.Icon(color="black")).add_to(map_accidents_carrefour_Grenoble)

## Comparaison avec Strasbourg : l'autre ville branchée Vélo

In [15]:
df_strasbourg_2023 = df_accidents_2023[df_accidents_2023['dep']=='67']
df_strasbourg_2022 = df_accidents_2022[df_accidents_2022['dep']=='67']

In [16]:
df_strasbourg_2023 = pd.merge(df_strasbourg_2023,df_vehicules_2023, how='inner',on='Num_Acc')
df_strasbourg_2022 = pd.merge(df_strasbourg_2022,df_vehicules_2022, how='inner',on='Num_Acc')

In [17]:
print(len(df_strasbourg_2023),len(df_strasbourg_2022))

100 121


## Affinage de l'analyse

## Fonction  (à affiner)

In [18]:
def Nombre_accidents(fichier_vehicule,fichier_caracteristique,département):
    df_vehicule = pd.read_csv(fichier_vehicule,sep=';')
    df_accident = pd.read_csv(fichier_caracteristique,sep=';')
    df_accident=df_accident[df_accident['dep']==département]
    df_accident = pd.merge(df_accident,df_vehicule,how='inner',on='Num_Acc')
    return len(df_accident)

# Grenoble

In [19]:
df_carrefour = pd.read_csv('amenagement_carrefour.csv',sep=',')
df_carrefour

,locali_id,geo_point_2d,carref_id,carref_datesupp,carref_datecre,carref_datemaj,accessi_desc,id
0,7,"45.1592442108994,5.73097602290016",250,NaN,2.000010e+13,NaN,Accessible,124
1,7,"45.1784747317108,5.72400680810226",255,NaN,2.000010e+13,NaN,Accessible,125
2,7,"45.1787899004849,5.72390867736167",254,NaN,2.000010e+13,NaN,Accessible,126
3,7,"45.1919814588737,5.72857005674412",302,NaN,2.000010e+13,NaN,Accessible,127
4,7,"45.187256332043,5.71100083692238",261,NaN,2.000010e+13,NaN,Accessible,128
...,...,...,...,...,...,...,...,...
1301,7,"45.1842581189286,5.74677811022395",1068,NaN,2.000010e+13,NaN,Pas de passage piéton,1283
1302,7,"45.1845794749506,5.7134104469574",1075,NaN,2.000010e+13,NaN,Pas de passage piéton,1284
1303,7,"45.1738004173595,5.74005400273794",1170,NaN,2.000010e+13,NaN,Pas de passage piéton,1285
1304,7,"45.1669120305242,5.73823523806341",1182,NaN,2.000010e+13,NaN,Pas de passage piéton,1286


In [20]:
df_carrefour['latitude'] = [element[0] for element in df_carrefour['geo_point_2d'].apply(lambda x : x.split(','))]
df_carrefour['longitude'] = [element[1] for element in df_carrefour['geo_point_2d'].apply(lambda x : x.split(','))]



In [21]:
df_carrefour['accessi_desc'].unique()

array(['Accessible', 'Inaccessible', 'Part. accessible',
       'Pas de passage piéton'], dtype=object)

In [22]:
df_carrefour.groupby('accessi_desc')['carref_id'].count().reset_index()

,accessi_desc,carref_id
0,Accessible,774
1,Inaccessible,194
2,Part. accessible,116
3,Pas de passage piéton,222


On prend un point qui va nous servir de centre

In [23]:
for index, row in df_carrefour[df_carrefour['accessi_desc']=='Accessible'].iterrows():
    folium.Marker([row['latitude'], row['longitude']],icon=folium.Icon(color="green")).add_to(map_accidents_carrefour_Grenoble)

for index, row in df_carrefour[df_carrefour['accessi_desc']=='Pas de passage piéton'].iterrows():
    folium.Marker([row['latitude'], row['longitude']],icon=folium.Icon(color="white")).add_to(map_accidents_carrefour_Grenoble)

for index, row in df_carrefour[df_carrefour['accessi_desc']=='Part. accessible'].iterrows():
    folium.Marker([row['latitude'], row['longitude']],icon=folium.Icon(color="orange")).add_to(map_accidents_carrefour_Grenoble)

for index, row in df_carrefour[df_carrefour['accessi_desc']=='Inaccessible'].iterrows():
    folium.Marker([row['latitude'], row['longitude']],icon=folium.Icon(color="red")).add_to(map_accidents_carrefour_Grenoble)

In [24]:
map_accidents_carrefour_Grenoble.save('carte_carrefour_accidents.html')

<iframe src="ma_carte.html" width="800" height="600"></iframe>

Ce qu'on remarque c'est que sur 10 accidents dans le centre ville élargit de Grenoble , il y en a 4 ou 5 qui se sont passés très proche d'un carrefour.
Ce qu'il s'est passé, nous manquons d'informations sur les conditions de cet accident mais le risque que ce soit du à l'agencement ou à la disposition des véhicules à proximité du carrefour doit très certainement jouer.
L'accessibilité du carrefour ne semble pas être un facteur déterminant de cause de l'accident.
Les données gps relatives aux carrefours exterieurs à Grenoble manque et ne couvrent pas les accidents en dehors de la zone du centre et hypercentre.

## Deuxième indicateur : comptage permanent vélo

In [25]:
import pandas as pd

df=pd.read_csv("BaseDeDonnées/comptages_velos_permanents.csv")
df_vf=df[df['territoire'] == 'Grenoble Alpes Metropole']

#print(df.info())
#print(df.head(3))
df_vf.head()

FileNotFoundError: [Errno 2] No such file or directory: 'BaseDeDonnées/comptages_velos_permanents.csv'

In [ ]:
# Afficher ou vérifier les premières lignes du nouveau DataFrame
colonnes=['nom_post', 'modes','localisation','type_compteur','type_axe','tmj_2019','tmj_2020','tmj_2021','tmj_2022']
colonnes_bis=['nom_post','localisation','type_axe','tmj_2019','tmj_2020','tmj_2021','tmj_2022']

print(df_vf[colonnes_bis].head(10))

#### Comme tous les compteurs n'ont pas été installés au même moment, il existe beaucoup de valeurs manquantes (NaN), il sera alors nécessaire de les ignorer au moment des analyses car il ne conviendrait pas de les remplacer par des zéros (fausseraient les résultats) de supprimer les lignes associées (trop nombreuses)
#### Ce qu'il faudrait c'est donc au moment de faire les analyses, passer outre ces valeurs vides (si l'on cherche a avoir des moyennes par années ou par ville par exemple)

## Troisième indicateur : Kilomètre piste cyclable

In [290]:
from shapely.geometry import Point
from shapely.geometry import LineString
import geopandas as gpd
from shapely.wkt import load

In [291]:
df_piste = pd.read_csv('pistes_cyclables.xls')

In [292]:
df_piste['lat'] = [element[0] for element in df_piste['geo_point_2d'].str.split(',')]
df_piste['long'] = [element[1] for element in df_piste['geo_point_2d'].str.split(',')]

In [293]:
df_piste['lat'] = df_piste['lat'].astype(object)
df_piste['long'] = df_piste['long'].astype(object)



In [294]:
df_piste['segment'] = df_piste['ogc_fid']

On prend notre point d'encrage de la map

In [295]:
map_piste_cyclable = folium.Map(location=[45.188529, 5.724524],
                                              zoom_start=14,
                                              titles = 'Map Piste Cyclable')

Transformation de geo_shape en json

In [296]:
df_piste['geo_shape'] = df_piste['geo_shape'].apply(lambda x: LineString(eval(x)['coordinates']))

In [297]:
df_piste['coordonnees']= df_piste['geo_shape'].apply(lambda x : list(x.coords)).to_list()


In [298]:
def reverse_coordonnees(liste):
  return [(coordonnees[1],coordonnees[0]) for coordonnees in liste ]

In [299]:
df_piste['coordonnees']= df_piste['coordonnees'].apply(lambda x: reverse_coordonnees(x))

In [301]:
for cords in df_piste['coordonnees']:
    folium.PolyLine(cords,color = 'red').add_to(map_piste_cyclable)

In [302]:
df_piste['lat'] = df_piste['lat'].astype(float)
df_piste['long'] = df_piste['long'].astype(object)

In [303]:
map_piste_cyclable #affichage de la carte

In [304]:
map_piste_cyclable.save('map_pistes_cyclables.html')

Explications

### Vérification des type LineString

## traffic routier

In [336]:
!pip install pyproj

In [337]:
import pyproj
from pyproj import Transformer

In [311]:
from lxml import etree

root = etree.parse('qtvDir.xml')
for elem in root.iter():
    print( elem.text)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
                    

                        

                            
83.333336

            
None
2025-01-18T14:42:00.000+01:00

                

                    

                        

                            
2100

                

                    

                        

                            
82.0

            
None
2025-01-18T14:42:00.000+01:00

                

                    

                        

                            
930

                

                    

                        

                            
73.0

            
None
2025-01-18T14:42:00.000+01:00

                

                    

                        

                            
1840

                

                    

                        

                            
80.0

            
None
2025-01-18T14:42:00.000+01:00

                

               

In [ ]:
## Code communes Grenoble
L_communes = [38185,38421,38563,38239] #Grenoble , Saint martin d'heres , Voiron , Moirans

In [356]:
df_traffic = pd.read_csv('tmja-2019.csv',sep=';')

In [357]:
#print(df_traffic.dtypes)
df_traffic = df_traffic[df_traffic['depPrD']==38]

In [358]:
df_traffic = df_traffic.dropna() # Il y en a très peu

In [329]:
df_traffic


,dateReferentiel,route,longueur,prD,depPrD,concessionPrD,absD,cumulD,xD,yD,...,absF,cumulF,xF,yF,zF,anneeMesureTrafic,typeComptageTrafic,typeComptageTrafic_lib,TMJA,ratio_PL
240,01/01/2019,A0007,10245,6,38,C,0,20364,"843170,65","6487932,34",...,245,30609,"841286,89","6478245,59",0,2019.0,1.0,Permanent horaire,75394.0,"19,41"
241,01/01/2019,A0007,7385,16,38,C,245,30609,"841286,89","6478245,59",...,630,37994,"841946,36","6471025,19",0,2019.0,1.0,Permanent horaire,76478.0,"19,27"
242,01/01/2019,A0007,2650,23,38,C,630,37994,"841946,36","6471025,19",...,280,40644,"842482,22","6468533,76",0,2019.0,1.0,Permanent horaire,69957.0,"19,38"
243,01/01/2019,A0007,28826,26,38,C,280,40644,"842482,22","6468533,76",...,106,69470,"847309,67","6442597,33",0,2019.0,1.0,Permanent horaire,69957.0,"19,38"
282,01/01/2019,A0007N,906,21,38,C,0,740,"840541,5","6499453,8",...,906,1646,"841061,47","6498731,41",0,2019.0,1.0,Permanent horaire,109036.0,"14,63"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3434,01/01/2019,N0087,1991,6,38,N,0,5676,"916815,45","6455970,08",...,0,7667,"918170,73","6457408,97",0,2019.0,1.0,Permanent horaire,79523.0,59
3435,01/01/2019,N0087,54,8,38,N,0,7667,"918170,73","6457408,97",...,54,7721,"918201,46","6457453,55",0,2019.0,1.0,Permanent horaire,78714.0,54
3436,01/01/2019,N0087,"57,8",8,38,N,54,7721,"918201,46","6457453,55",...,"111,8","7778,8","918232,57","6457502,42",0,2019.0,1.0,Permanent horaire,78714.0,54
3437,01/01/2019,N0087,"588,2",8,38,N,"111,8","7778,8","918232,57","6457502,42",...,700,8367,"918422,14","6458056,41",0,2019.0,1.0,Permanent horaire,78714.0,54


In [359]:
def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

In [346]:
df_traffic.dtypes

,0
dateReferentiel,object
route,object
longueur,object
prD,int64
depPrD,int64
concessionPrD,object
absD,object
cumulD,object
xD,object
yD,object


In [360]:
df_traffic['xD']=df_traffic['xD'].str.replace(',','.')
df_traffic['xF']=df_traffic['xF'].str.replace(',','.')
df_traffic['yD']=df_traffic['yD'].str.replace(',','.')
df_traffic['yF']=df_traffic['yF'].str.replace(',','.')



df_traffic['xD']=df_traffic['xD'].astype(float)
df_traffic['yD']=df_traffic['yD'].astype(float)
df_traffic['xF']=df_traffic['xF'].astype(float)
df_traffic['yF']=df_traffic['yF'].astype(float)

In [354]:
df_traffic.dtypes

,0
dateReferentiel,object
route,object
longueur,object
prD,int64
depPrD,int64
concessionPrD,object
absD,object
cumulD,object
xD,float64
yD,object


In [361]:
df_traffic['coordonnees_gps_debut'] = [xy_to_lonlat(element_x,element_y) for element_x,element_y in zip(df_traffic['xD'],df_traffic['yD'])]

<ipython-input-359-b08330dd5b53>:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-359-b08330dd5b53>:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-359-b08330dd5b53>:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-359-b08330dd5b53>:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-359-b08330dd5b53>:4: FutureWarning: This function is deprecated. 

In [363]:
df_traffic['coordonnees_gps_fin'] = [xy_to_lonlat(element_x,element_y) for element_x,element_y in zip(df_traffic['xF'],df_traffic['yF'])]

<ipython-input-359-b08330dd5b53>:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-359-b08330dd5b53>:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-359-b08330dd5b53>:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-359-b08330dd5b53>:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-359-b08330dd5b53>:4: FutureWarning: This function is deprecated. 

In [365]:
df_traffic['coordonnees'] = [(element_d,element_f) for element_d,element_f in zip(df_traffic['coordonnees_gps_debut'],df_traffic['coordonnees_gps_fin'])]

In [371]:
df_traffic['coordonnees'] = df_traffic['coordonnees'].apply(lambda x : reverse_coordonnees(x))

In [372]:
map_route = folium.Map(location=[45.188529, 5.724524],
                                              zoom_start=14,
                                              titles = 'Map Piste Cyclable')

In [373]:
for cords in df_traffic['coordonnees']:
    folium.PolyLine(cords,color = 'blue').add_to(map_route)

In [374]:
map_route

On parle de df_piste dans cette partie

In [83]:
# Fonction pour vérifier le type de géométrie
def is_linestring(geo_shape):
    try:
        shape = json.loads(geo_shape)
        return shape['type'] == 'LineString'
    except:
        return False

# Vérifier si toutes les géométries sont des LineString
df['is_linestring'] = df['geo_shape'].apply(is_linestring)

# Résumé des résultats
print("Résumé des types de géométries :")
print(df['is_linestring'].value_counts())

# Identifier les géométries non conformes, s'il y en a
non_linestring_rows = df[~df['is_linestring']]
if not non_linestring_rows.empty:
    print("Lignes avec des géométries non conformes :")
    print(non_linestring_rows)
else:
    print("Toutes les géométries sont des LineString.")

Résumé des types de géométries :
is_linestring
True    1202
Name: count, dtype: int64
Toutes les géométries sont des LineString.


### Calcul des distances de la piste cyclable

In [ ]:
# Fonction pour calculer la distance d'une piste cyclable à partir de la colonne geo_shape
def calculate_distance(geo_shape):
    try:
        # Charger les coordonnées GeoJSON
        shape = json.loads(geo_shape)
        if shape['type'] == 'LineString':
            coordinates = shape['coordinates']
            # Calculer la distance totale entre les points consécutifs
            distances = [
                geodesic(coordinates[i], coordinates[i+1]).kilometers
                for i in range(len(coordinates) - 1)
            ]
            return sum(distances)
    except:
        return None  # Retourner None si un problème survient

# Ajouter une colonne pour les distances
df['distance_km'] = df['geo_shape'].apply(calculate_distance)

# Enregistrer le DataFrame avec la nouvelle colonne dans un nouveau fichier CSV
df.to_csv('fichier_avec_distances.csv', index=False)

# Afficher les premières lignes pour vérifier
print(df.head())

# Étude comparative étudiants UGA + INP

In [ ]:
df_uga = pd.read_csv('BaseDeDonnées/Etude_UGA/déplacements_UGA_output.csv',sep=';')

In [ ]:
df_uga

In [ ]:
data_car